### 6.2.2 Making a more efficient and robust implementation

For more general use, there are some pitfalls that should be fixed in an improved version of the code.

An example may illustrate wht the problem is: let us solve $tanh(x) = 0$, which has solution $x=0$.
With $|x_0| \leq 1.08$ everything works fine.

Adjusting $x_0$ slightly to 1.09 gives division by zero!

In [3]:
import numpy as np

print('----------------------------------------------')
print('             x             f(x) = tanh(x)  ')
print('----------------------------------------------')

def naive_Newton(f, dfdx, x, epx):
    while abs(f(x)) > epx:
        x = x -float(f(x))/dfdx(x)
        print('% 20.10f, % 20.10f' % (x, f(x)))
    return x

def f(x):
    return np.tanh(x)

def dfdx(x):
    return 1-(np.tanh(x))**2

#print(naive_Newton(f, dfdx, 1.08, 0.001))
print(naive_Newton(f, dfdx, 1.09, 0.001))

----------------------------------------------
             x             f(x) = tanh(x)  
----------------------------------------------
       -1.0933161820,        -0.7980853072
        1.1049035432,         0.8022534801
       -1.1461555079,        -0.8164764710
        1.3030326182,         0.8625018187
       -2.0649230024,        -0.9683385739
       13.4731428006,         1.0000000000
-126055913647.1373901367,        -1.0000000000
                 inf,         1.0000000000
                 nan,                  nan
nan


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  if __name__ == '__main__':
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


The original naive_Newton function is that it calls the $f(x)$ function twice as many times as necessary. This extra work is of no concern when $f(x)$ is fast to evaluate, but in large-scale industrial software, one call to $f(x)$ might take hours or days, and then removing unnecessary calls is important.

The solution in our function is to sotre the call $f(x)$ in a variable (f_value) and reuse the value insted of making a new call $f(x)$.

To summarize, we want to write an improved function for rimplementing Newton's method where we

- avoid division by zero
- allow a maximum number of iterations
- avoid the extra evaluation to $f(x)$

A more robust and efficient version of the function, inserted in a complete program for solving $x^2 -9 =0$, is listed below.

In [2]:
import numpy as np

def Newton(f, dfdx, x, eps):
    f_value = f(x)
    iteration_counter = 0
    while abs(f_value) > eps and iteration_counter < 100:
        try:
            x = x -float(f_value)/dfdx(x)
        except ZeroDivisionError:
            print("Error! - derivative zero for x = ", x)
            sys.exit(1)  # Abort with error / Sys.exit(0) means succesfully operating
            
        f_value = f(x)
        iteration_counter += 1
        
    # Here, either a solution is found, or too many iterations
    if abs(f_value) > eps:
        iteration_counter = -1
    return x, iteration_counter

def f(x):
    return x**2 - 9
#    return np.tanh(x)

def dfdx(x):
    return 2*x
#    return 1-(tahh(x))**2

solution, no_iterations = Newton(f, dfdx, x=1000, eps=1.0e-6)

if no_iterations > 0:  # Solution found
    print("Number of function calls: %d" % (1 + 2*no_iterations))  # Newtwon(x) 1 + f(x),dfdx 2*n = 2n +1
    print("A solution is %f" % (solution))
else:
    print("Solution not found!")

Number of function calls: 25
A solution is 3.000000


Newton's method requires the analytical expression for the derivative $f'(x)$. Derivation of $f'(x)$ is not always a reliable process by hand if $f(x)$ is a complicated function. However, Python has the symbolic package SymPy, which we may use to create the required $dfdx$ function. In our sample problem, the recipe goes as follow:

In [5]:
from sympy import *
x = symbols('x')              # define x as a mathematical symbl

#f_expr = x**2 -9             # symbolic expression for f(x)
f_expr = x**3 -2*x**2 - 10*x

dfdx_expr = diff(f_expr, x)  # compute f'(x) symbolically
# Turn f_expr and dfdx_expr into plain Python functions

f = lambdify([x],            # argument to f
            f_expr)          # symbolic expression to be evaluted.
                             # f(x) = 2*x, lambdify(x,f(x)), lambdify(5) = 10 
dfdx = lambdify([x], dfdx_expr)

print(dfdx(5))               # will print 10

45
